recall

* when $q(z) = \prod_i q_i(z_i)$ is factorized, then the minimizer of $d_{KL}(p || q)$ is $E_q[U(Z)] = E_p[U(Z)]$
* optimization criterion (e.g., calculating the marginals) is hard

## gaussian process

$f \sim GP(m(\cdot), K(\cdot, \cdot))$  
$f \sim \mathcal{N}(m(x), K(x, x^\top))$

$x = (x_1, ..., x_N)$  
$f = (f(x_1), ..., f(x_N))$

typically don't observe $f$ (the function itself) but $t$ which depend on $f$

regression case

* $p(t_i | f_i) = \mathcal{N}(f_i, \beta^{-1})$
* assume $m = 0$, C_N = K(x, x^\top) + \beta^{-1} I$
* $ t \sim \mathcal{N}(0, C_N)$

classification

* $p(t_i | f_i) = Bernoulli(\sigma(f_i))$ (logistic regression) or  
$p(t_i | f_i) = \sigma(t_i f_i)$ or  
$p(t_i | f_i) = \Phi(t_i f_i)$ (probit regression)

#### setup for EP

* consider true posterior $\propto p(\theta) \prod p(x_i | \theta)$

* adjust posterior to $q(\theta) \propto \prod_{i=0}^N \tilde{f}_i(\theta)$

* can't minimize KL-divergence directly
* EP instead iteratively improves $\tilde{f}_i$
* not guaranteed to actually minimize KL-divergence
* no convergence guarantee

### EP for GP classification

$p(f, t) = p(f) p(t | f) = \mathcal{N}(f | 0, K) \prod_i p(t_i | f_i)$

$q(f) \propto p(f) \prod_i \tilde{g}_i(f_i)$

$\tilde{g}_i(f_i) = z_i \mathcal{N}(f_i | \tilde{\mu}_i, \tilde{s}_i)$

1. $\prod \tilde{g}_i (f_i) \propto \mathcal{N}(f | \tilde{\mu}, \tilde{\Sigma})$  
$\tilde{\mu} = \begin{bmatrix} \tilde{\mu}_1 & \cdots & \tilde{\mu}_N \end{bmatrix}^\top$  
$\tilde{\Sigma} = diag(\tilde{s}_i)$  
$q(f) \sim \mathcal{\mu, \Sigma})$  
$\Sigma = (K^{-1} \tilde{S}^{-1})^{-1}$  
$\mu = \Sigma \tilde{s}^{-1} \tilde{\mu}$

2. $q^{(-j)}(\theta) = \frac{q^{(old)}(\theta)}{f_j(\theta)}$ which is a multivariate normal of $(f_1, ..., f_N)$ divided by univariate normal of $f_j$  
so we get $q^{(old)}(f) = q^{(old)}(f_j) q^{(old)}(f^{(-j)} | f_j)$  
and $q^{(-j)}(\theta) = \frac{q^{(old)}(f^{(-j)} | f_j) q^{(old)}(f_j)}{\tilde{g}_j(f_j)}$  
so all we need is 1D division  
$q^{(old)}(f) = \mathcal{N}(f | \mu, \Sigma)$  
then $q^{(old)}(f_j) = \mathcal{N}(\mu_j, \Sigma_{jj})$  
$q^{(-j)}(f) \propto q^{(old)}(f^{(-j)} | f_j) \frac{\mathcal{N}(f_j | \mu_j, \Sigma_{jj})}{\mathcal{N}(f_j | \tilde{\mu}_j, \tilde{s}_j)}$  
so $q^{(-j)}(f_j) = \mathcal{N}(f_j | \mu^{(-j)}, s^{(-j)})$
    * $s^{(-j)} = (\Sigma_{jj}^{-1} - \tilde{s}_j^{-1})^{-1}$
    * $\mu^{(-j)} = s^{(-j)} (\mu_j / \Sigma_{jj} - \tilde{\mu}_j / \tilde{s}_j)$

3. $\hat{p}(\theta) = z_j^{-1} f_j(\theta) q^{(-j)}(\theta)$  
$z_j = \int q^{(-j)}(f) p(t_j | f_j) df = \int q^{(-j)}(f_j) p(t_j | f_j) d f_j$  
can calculate this sometimes (e.g., logistic regression), or if not, use sampling

4. $q^{(new)}(\theta) = \arg\min d_{KL}(\hat{p}(\theta) || q^{(new)}(\theta))$  
perform moment matching $E_{\hat{p}}[f_j] = E_{q^{(new)}}[f_j]$ and $E_{\hat{p}}[f_j^2] = E_{q^{(new)}}[f_j^2]$  
$q^{(new)}(f_j) = \mathcal{N}(f_j | \hat{\mu}_j, \hat{s}_j)$  
$q^{(new)}(f) = q^{(-j)}(f^{(-j)} | f_j) q^{(new)}(f_j)$

5. $\tilde{f}_j^{(new)} = z_j \frac{q^{(new)}(\theta)}{q^{(-j)}(\theta)}$  
$\tilde{g}_j^{(new)}(f_j) = z_j \frac{q^{(new)}(f_j)}{q^{(-j)}(f_j)}$  
$= \frac{z_j}{A} \mathcal{N}(f_j | \tilde{\mu}_j^{(new)}, \tilde{s}_j^{(new)})$
    * $\tilde{s}_j^{(new)} = (\hat{s}_j^{-1} - (s^{(-j)})^{-1})^{-1}$
    * $\tilde{\mu}_j^{(new)} = \tilde{s}_j^{(new)} (\hat{\mu}_j / \hat{s}_j - \mu^{(-j)} / s^{(-j)})$
    * $A = \mathcal{N}(\hat{\mu}_j | \mu^{(-j)}, \hat{s}_j + s^{(-j)})$

for step 1 of next iteration, need explicit form of $q^{(new)}(f)$  
just calculate by $\propto \prod \tilde{g}_j(f_j) p(f)$

### EP for factor graphs

brief reminder/example of BP messages

![](loopy-graph.png)

want to find $\mu_{f_b \to x_1}(x_1)$  
$= \sum_{x_2, x_3} f_b(x_1, x_2, x_3) \mu_{x_3 \to f_b}(x_3) \mu_{x_2 \to f_b}(x_2)$

$\mu_{x_3 \to f_b}(x_3) = \mu_{f_c \to x_3}(x_3)$

$\mu_{x_2 \to f_b}(x_2) = \mu_{f_c \to x_2}(x_2) \times \mu_{f_a \to x_2}(x_2)$

then $\mu_{f_b \to x_1} (x_1) = \sum_{x_2, x_3} f_b(x_1, x_2, x_3) 
\times \mu_{f_c \to x_3}(x_3) 
\times (\mu_{f_b \to x_2}(x_2) \times \mu_{f_a \to x_2}(x_2))$

factor graph: $p(\theta) \propto \prod_i f_i (c_i)$

(clutter problem: $\prod f(\text{all } \theta)$)

(GP: $prior(\text{all } \theta) \prod f_i (\theta_i)$)

EP posterior: $q(\theta) \propto \prod_i \tilde{f}_i (c_i)$ 

in this example, use factored representation:  
$q(\theta) = \prod_k q(\theta_k)$

then $q(\theta) \propto \prod_i \prod_{k \in c_i} \tilde{f}_{ik} (\theta_k)$

$f_a(x_1, x_2, x_4) f_b(x_1, x_2, x_3) f_c(x_2, x_3) \approx \tilde{f}_{a1}(x_1) \tilde{f}_{a2}(x_2) \tilde{f}_{a4}(x_4) \tilde{f}_{b1}(x_1) \tilde{f}_{b2}(x_2) \tilde{f}_{b3}(x_3) \tilde{f}_{c2}(x_2) \tilde{f}_{c3}(x_3)$

1. calculate $q(\theta) = \prod \tilde{f}_i(\theta)$  
normally we calculate compact form $f \sim q(\theta)$,  
but $(*)$ will deal with it later  
just use: $q(\theta) \propto \prod_i \prod_{k \in c_i} \tilde{f}_{ik} (\theta_k)$

2. calculate $q^{(-j)}(\theta)$  
do not normalize for now  
$(*) = \prod_{i \neq j} \prod_k \tilde{f}_{ik}(\theta_k)$  
e.g., $q^{(-f_b)} = \tilde{f}_{a1} \tilde{f}_{a2} \tilde{f}_{a4} \tilde{f}_{c2} \tilde{f}_{c3}$

3. calculate $\hat{p}(\theta) \propto q^{(-j)}(\theta) f_j(c_j)$  
normally calculate $z_j$ and let $\hat{p}(\theta) = z_j^{-1} q^{(-j)}(\theta) f_j (c_j)$  
but $(*)$

4. calculate $q^{(new)}(\theta) = \arg\min d_{KL}(\hat{p}(\theta) || q^{(new)}(\theta))$  
where $q^{(new)}(\theta) = \prod_k q^{(new)}(\theta_k)$  
since $q^{(new)}$ is factorized, $q^{(new)}(\theta_k) = \hat{p}(\theta_k)$  
$\implies$ need to calculate $\hat{p}(\theta_k)$  
    * $\theta_k \not\in c_j$  
    given $f_b(x_1, x_2, x_3)$, $x_4 \in \{x_1, x_2, x_4\}$  
    $\hat{p}(\theta) \propto \tilde{f}_{a1} \tilde{f}_{a2} \tilde{f}_{a4} \tilde{f}_{c2} \tilde{f}_{c3} f_b(x_1, x_2, x_3)$  
    $\hat{p}(\theta_k) = q^{(-j)}(\theta_k) = q^{(old)}(\theta_k)$  
    * $\theta_k \in c_j$  
    let $c_j = \theta_k, \theta_{l_1}, ..., \theta_{l_m}$  
    $\hat{p}(\theta_k) = \sum_{\text{all but } \theta_k} \hat{p}(\theta)$  
    $= (\sum_{\theta_l \in c_j} f_j(c_j) \prod \hat{f}(\theta_l)) (\sum_{\theta_l \not\in c_j} \prod \tilde{f}_{\cdot_l}(\theta_l))$  
    first term: variables in $c_j$, second term: variables not in $c_j$  
    second term sums/integrates out to a constant  
    * then $\hat{p}(x_1) \propto \tilde{f}_{a1} \sum_{x_2, x_3} \tilde{f}_{a2} \tilde{f}_{c2} \tilde{f}_{c3} f_b(x_1, x_2, x_3) \sum_{x4} \tilde{f}_{a4}$  
    the last summation over $x_4$ goes to a constant that we can ignore  
    we are left with two parts
        * $\tilde{f}_{a1}$ only depends on $x_1$
        * $\sum_{x_2, x_3} \cdots$ 
    * $\hat{p}(\theta_k) \propto \prod_{x \neq j, k \in c_j}\tilde{f}_{ik}(\theta_k) \sum_{\theta_{l_1}, ..., \theta_{l_m}} f_j(c_j) \prod_{l_m} \prod_{i \neq j s.t. l_m \in c_i} \tilde{f}_{i{l_m}}(\theta_{l_m})$  
    * so $q^{(new)}(\theta) = \prod_k q^{(new)}(\theta_k) \propto \prod_k \hat{p}(\theta_k)$ of the form above
    * no compact/normalized form but $(*)$
5. want to calculate $\tilde{f}_j(c_j) = \prod_{k \in c_j} \tilde{f}_{jk}(\theta_k)$  
typically $\tilde{f}_j(c_j) = q^{(new)}(\theta) / q^{(-j)}(\theta)$  
variables not in $c_j$ are not affected or changed  
$q^{(-f_b)} = \tilde{f}_{a1} \tilde{f}_{a2} \tilde{f}_{a4} \tilde{f}_c{2} \tilde{f}_{c3}$  
$q^{(new)}(x_1) \propto \tilde{f}_{a1} \sum_{x_2, x_3} \tilde{f}_{a2} \tilde{f}_{c2} \tilde{f}_{c3} f_b(x_1, x_2, _x3)$  
so $q^{(new)} / q^{(-f_b)}$ has stuff canceling out  
$\tilde{f}_{jk}(\theta_k) \propto \sum_{\theta_{l_1}, ..., \theta_{l_m}} f_j(c_j) \prod_{l_m} \prod_{i \neq j, l_m \in c_i} \hat{f}_{i l_m} (\theta_{l_m})$  
$\tilde{f}_{jk} = \mu_{c_j \to \theta_k}(\theta_k)$
$\propto \sum_{\theta_1, ..., \theta_k} f_j(c_j) \prod_{l_m} \prod_{i \neq j, l_m \in c_i} \mu_{c_i \to l_m} (\theta_{l_m})$  
$\prod_{i \neq j, l_m \in c_i} \mu_{c_i \to l_m}(\theta_{l_m})$ is the product of $\mu_{c_i \to l_m}$ from all factors except $c_j$ which is equivalent to $\mu_{l_m \to c_j}(\theta_{l_m})$  

update equations of sub-factors in EP when using factored $q(\theta)$ is identical to messages in BP